<a href="https://colab.research.google.com/github/rfasich24/Skripsi-E-Nose/blob/main/Memecah_Dataset_E_Nose_interval_di_Interval_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

untuk mengubah delimiter

In [ ]:
import pandas as pd
import glob
import os

# --- PENGATURAN ---
# Tentukan folder tempat file CSV Anda berada.
# Jika Anda menjalankan ini di Google Colab/Jupyter dan file berada di root (utama), gunakan '.'
FOLDER_PATH = '.'  # Ganti dengan path folder Anda jika tidak berada di direktori saat ini
# ------------------

# Mencari semua file dengan ekstensi .csv di folder yang ditentukan
csv_files = glob.glob(os.path.join(FOLDER_PATH, '*.csv'))

if not csv_files:
    print(f"❌ TIDAK DITEMUKAN: Tidak ada file CSV di folder '{FOLDER_PATH}'.")
else:
    print(f"✅ DITEMUKAN {len(csv_files)} file CSV. Memulai konversi...")

    # Loop melalui setiap file yang ditemukan
    for file_path in csv_files:
        print(f"\n--- Memproses file: {file_path}")

        df = None

        # 1. Coba baca file dengan delimiter KOMA (default CSV standar)
        try:
            # Menggunakan engine 'python' dan sep=','
            temp_df = pd.read_csv(file_path, sep=',', engine='python', encoding='utf-8')

            # Cek apakah hanya memiliki 1 kolom (indikasi gagal deteksi koma, mungkin aslinya titik koma)
            if temp_df.shape[1] > 1:
                df = temp_df
                delimiter_awal = 'KOMA'

        except Exception as e:
            # Lanjutkan ke percobaan kedua jika gagal
            pass

        # 2. Jika pembacaan koma gagal, coba baca file dengan delimiter TITIK KOMA
        if df is None:
            try:
                # Menggunakan sep=';'
                temp_df = pd.read_csv(file_path, sep=';', engine='python', encoding='utf-8')

                # Pastikan data terbaca lebih dari 1 kolom
                if temp_df.shape[1] > 1:
                    df = temp_df
                    delimiter_awal = 'TITIK KOMA'

            except Exception as e:
                print(f"   ⚠️ GAGAL BACA: Gagal membaca file dengan KOMA maupun TITIK KOMA. Melewati file ini.")
                continue

        # 3. Jika berhasil dibaca, SIMPAN KEMBALI dengan delimiter KOMA
        if df is not None:
            # Gunakan file_path yang sama untuk menimpa file asli
            # sep=',' adalah nilai default, index=False mencegah penulisan kolom indeks/nomor baris
            df.to_csv(file_path, sep=',', index=False, encoding='utf-8')
            print(f"   👍 BERHASIL: Delimiter berhasil dikonversi dari '{delimiter_awal}' dan disimpan kembali menggantikan file utama.")
        else:
             print(f"   ⚠️ GAGAL BACA: File terbaca sebagai 1 kolom. Mungkin ada masalah format di '{file_path}'. Melewati file ini.")

print("\n--- SEMUA PROSES SELESAI ---")

✅ DITEMUKAN 15 file CSV. Memulai konversi...

--- Memproses file: ./Sample 4 Grade 4.csv
   👍 BERHASIL: Delimiter berhasil dikonversi dari 'KOMA' dan disimpan kembali menggantikan file utama.

--- Memproses file: ./Sample 4 Grade 2.csv
   👍 BERHASIL: Delimiter berhasil dikonversi dari 'KOMA' dan disimpan kembali menggantikan file utama.

--- Memproses file: ./Sample 3 Grade 2.csv
   👍 BERHASIL: Delimiter berhasil dikonversi dari 'TITIK KOMA' dan disimpan kembali menggantikan file utama.

--- Memproses file: ./Sample 5 Grade 4.csv
   👍 BERHASIL: Delimiter berhasil dikonversi dari 'KOMA' dan disimpan kembali menggantikan file utama.

--- Memproses file: ./Sample 5 Grade 2.csv
   👍 BERHASIL: Delimiter berhasil dikonversi dari 'KOMA' dan disimpan kembali menggantikan file utama.

--- Memproses file: ./Sample 4 Grade 3.csv
   👍 BERHASIL: Delimiter berhasil dikonversi dari 'KOMA' dan disimpan kembali menggantikan file utama.

--- Memproses file: ./Sample 3 Grade 3.csv
   👍 BERHASIL: Delimite

untuk mengsplit data

In [ ]:
import pandas as pd
import glob
import os
import zipfile # Pustaka untuk kompresi ZIP
from google.colab import files

# ===========================================================
# KONFIGURASI
# ===========================================================

POLA_FILE_INPUT = '*.csv'
BATAS_COUNTER = 1000
NAMA_FILE_ZIP = 'hasil_split_numerik_csv.zip' # Nama file ZIP output
file_output_terbuat = []

print("--- Skrip Pemrosesan Kolektif CSV (ZIP & Download Otomatis) ---")
print(f"Batas Counter: {BATAS_COUNTER}")
print(f"Output akan dikompresi ke dalam file: {NAMA_FILE_ZIP}")
print("-" * 70)

# ===========================================================
# PERSIAPAN DAN PENCARIAN FILE
# ===========================================================

daftar_file_input = glob.glob(POLA_FILE_INPUT)
daftar_file_input_bersih = [f for f in daftar_file_input if not f.startswith(('SPLIT_', NAMA_FILE_ZIP))]

print(f"Ditemukan {len(daftar_file_input_bersih)} file CSV untuk diproses: {daftar_file_input_bersih}")
print("-" * 70)

# ===========================================================
# LOOP PEMROSESAN FILE & KONVERSI NUMERIK
# ===========================================================

if not daftar_file_input_bersih:
    print("❌ ERROR: Tidak ada file CSV yang ditemukan untuk diproses.")
else:
    for file_input in daftar_file_input_bersih:
        print(f"\n➡️ MEMPROSES FILE: {file_input}")

        try:
            # A. Memuat File
            df = pd.read_csv(file_input)

            # B. Konversi Data ke Numerik
            print("   - 🔄 Mengubah semua kolom data menjadi numerik...")
            df_converted = df.apply(pd.to_numeric, errors='coerce')

            if 'Counter' in df_converted.columns:
                 # Pastikan Counter adalah Integer yang mendukung nilai NaN (Int64)
                df_converted['Counter'] = df_converted['Counter'].astype('Int64', errors='ignore')

            df = df_converted.copy()

            # C. Pemeriksaan Kolom
            if 'Counter' not in df.columns:
                print(f"   - ⚠️ Peringatan: Kolom 'Counter' tidak ditemukan. Melewati file ini.")
                continue

            # D. Pemisahan Data Berdasarkan 'Counter'
            df_part_1 = df[df['Counter'] <= BATAS_COUNTER].copy()
            df_part_2 = df[df['Counter'] > BATAS_COUNTER].copy()

            # E. Penamaan dan Penyimpanan File Output
            nama_dasar = os.path.basename(file_input)
            file_output_1 = f"SPLIT_0_{BATAS_COUNTER}_{nama_dasar}"
            file_output_2 = f"SPLIT_{BATAS_COUNTER+1}_AKHIR_{nama_dasar}"

            # Menyimpan Part 1
            if not df_part_1.empty:
                df_part_1.to_csv(file_output_1, index=False)
                print(f"   - ✅ Part 1 ({len(df_part_1)} baris) disimpan ke '{file_output_1}'")
                file_output_terbuat.append(file_output_1)
            else:
                print(f"   - ⚠️ Part 1 kosong. File '{file_output_1}' tidak dibuat.")

            # Menyimpan Part 2
            if not df_part_2.empty:
                df_part_2.to_csv(file_output_2, index=False)
                print(f"   - ✅ Part 2 ({len(df_part_2)} baris) disimpan ke '{file_output_2}'")
                file_output_terbuat.append(file_output_2)
            else:
                print(f"   - ⚠️ Part 2 kosong. File '{file_output_2}' tidak dibuat.")

        except Exception as e:
            print(f"   - ❌ Gagal memproses {file_input}. Error: {e}")

# ===========================================================
# KOMPRESI ZIP DAN UNDUH OTOMATIS
# ===========================================================
print("\n" + "=" * 70)
print(f"TOTAL {len(file_output_terbuat)} FILE CSV BARU DIBUAT.")

if file_output_terbuat:
    print(f"📦 Mengkompresi {len(file_output_terbuat)} file ke dalam {NAMA_FILE_ZIP}...")

    # Membuat file ZIP dan menambahkan semua file hasil ke dalamnya
    with zipfile.ZipFile(NAMA_FILE_ZIP, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for filename in file_output_terbuat:
            zipf.write(filename)

    print(f"✅ Kompresi selesai. Sekarang mengunduh file {NAMA_FILE_ZIP}...")

    # Unduh file ZIP
    try:
        files.download(NAMA_FILE_ZIP)
        print(f"⬇️ Memicu unduhan untuk '{NAMA_FILE_ZIP}'. Periksa folder Downloads Anda.")
    except Exception as e:
        print(f"❌ Gagal mengunduh {NAMA_FILE_ZIP}. Error: {e}")

    # Opsional: Hapus file CSV yang tidak dibutuhkan lagi untuk menghemat ruang Colab
    # print("🗑️ Menghapus file CSV split (opsional)...")
    # for filename in file_output_terbuat:
    #     os.remove(filename)

print("PROSES KOLEKTIF SELESAI.")

--- Skrip Pemrosesan Kolektif CSV (ZIP & Download Otomatis) ---
Batas Counter: 1000
Output akan dikompresi ke dalam file: hasil_split_numerik_csv.zip
----------------------------------------------------------------------
Ditemukan 15 file CSV untuk diproses: ['Sample 4 Grade 4.csv', 'Sample 4 Grade 2.csv', 'Sample 3 Grade 2.csv', 'Sample 5 Grade 4.csv', 'Sample 5 Grade 2.csv', 'Sample 4 Grade 3.csv', 'Sample 3 Grade 3.csv', 'Sample 5 Grade 3.csv', 'Sample 3 Grade 4.csv', 'Sample 2 Grade 2.csv', 'Sample 2 Grade 3.csv', 'Sample 1 Grade 2.csv', 'Sample 1 Grade 3.csv', 'Sample 2 Grade 4.csv', 'Sample 1 Grade 4.csv']
----------------------------------------------------------------------

➡️ MEMPROSES FILE: Sample 4 Grade 4.csv
   - 🔄 Mengubah semua kolom data menjadi numerik...
   - ✅ Part 1 (993 baris) disimpan ke 'SPLIT_0_1000_Sample 4 Grade 4.csv'
   - ✅ Part 2 (20443 baris) disimpan ke 'SPLIT_1001_AKHIR_Sample 4 Grade 4.csv'

➡️ MEMPROSES FILE: Sample 4 Grade 2.csv
   - 🔄 Mengubah semua

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Memicu unduhan untuk 'hasil_split_numerik_csv.zip'. Periksa folder Downloads Anda.
PROSES KOLEKTIF SELESAI.


Mengambil nilai interval 301 - 600

In [ ]:
import pandas as pd
import glob
import os
import zipfile
from google.colab import files

# ===========================================================
# KONFIGURASI
# ===========================================================

POLA_FILE_INPUT = '*.csv'
NAMA_FILE_ZIP = 'hasil_filter_time_csv.zip'
file_output_terbuat = []

# Parameter Filter Interval (Kelipatan 1000)
INTERVAL_START = 301
INTERVAL_END = 600   # **DIUBAH MENJADI 600**
INTERVAL_GRUP = 1000

print("--- Skrip Modifikasi CSV (Filter Interval & Kolom Time) ---")
print(f"Filter Counter: [1000n + {INTERVAL_START}, 1000n + {INTERVAL_END}]")
print("-" * 70)

# ===========================================================
# PERSIAPAN DAN PENCARIAN FILE
# ===========================================================

daftar_file_input = glob.glob(POLA_FILE_INPUT)
daftar_file_input_bersih = [f for f in daftar_file_input if not f.startswith(('FILTERED_', NAMA_FILE_ZIP))]

print(f"Ditemukan {len(daftar_file_input_bersih)} file CSV untuk diproses: {daftar_file_input_bersih}")
print("-" * 70)

# ===========================================================
# LOOP PEMROSESAN FILE & KONVERSI NUMERIK
# ===========================================================

if not daftar_file_input_bersih:
    print("❌ ERROR: Tidak ada file CSV yang ditemukan untuk diproses.")
else:
    for file_input in daftar_file_input_bersih:
        print(f"\n➡️ MEMPROSES FILE: {file_input}")

        try:
            # A. Memuat File
            df = pd.read_csv(file_input)

            # B. Konversi Data ke Numerik
            print("   - 🔄 Mengubah semua kolom data menjadi numerik...")
            df_converted = df.apply(pd.to_numeric, errors='coerce')

            if 'Counter' in df_converted.columns:
                df_converted['Counter'] = df_converted['Counter'].astype('Int64', errors='ignore')

            df = df_converted.copy()

            # C. Pemeriksaan Kolom
            if 'Counter' not in df.columns:
                print(f"   - ⚠️ Peringatan: Kolom 'Counter' tidak ditemukan. Melewati file ini.")
                continue

            print(f"   - Berhasil dimuat. Total baris awal: {len(df)}")

            # D. LOGIKA FILTER INTERVAL (301-600 di setiap 1000)

            print(f"   - 🔍 Menerapkan filter Counter: [1000n + {INTERVAL_START}, 1000n + {INTERVAL_END}]...")

            # 1. Hitung sisa (modulus) Counter dibagi 1000.
            df['Counter_Mod'] = df['Counter'] % INTERVAL_GRUP

            # 2. Terapkan kondisi filter: Modulus harus >= 301 dan <= 600.
            kondisi_filter = (df['Counter_Mod'] >= INTERVAL_START) & \
                             (df['Counter_Mod'] <= INTERVAL_END)

            df_filtered = df[kondisi_filter].copy()

            # Hapus kolom bantuan 'Counter_Mod'
            df_filtered = df_filtered.drop(columns=['Counter_Mod'])

            print(f"   - ✅ Filter selesai. Total baris setelah filter: {len(df_filtered)}")

            if df_filtered.empty:
                print(f"   - ⚠️ Setelah filter, data kosong. Melewati penyimpanan.")
                continue

            # E. TAMBAH KOLOM "TIME"

            # Sisipkan kolom 'Time' setelah 'Counter' (indeks 1)
            df_filtered.insert(1, 'Time', range(1, 1 + len(df_filtered)))
            print(f"   - ⏱️ Kolom 'Time' ditambahkan (1 hingga {len(df_filtered)}).")

            # F. Penamaan dan Penyimpanan File Output

            nama_dasar = os.path.basename(file_input)
            file_output = f"FILTERED_{nama_dasar}"

            # Menyimpan data yang sudah dimodifikasi
            df_filtered.to_csv(file_output, index=False)
            print(f"   - ✅ Data tersimpan ke '{file_output}'")
            file_output_terbuat.append(file_output)

        except Exception as e:
            print(f"   - ❌ Gagal memproses {file_input}. Error: {e}")

# ===========================================================
# KOMPRESI ZIP DAN UNDUH OTOMATIS
# ===========================================================
print("\n" + "=" * 70)

if file_output_terbuat:
    print(f"TOTAL {len(file_output_terbuat)} FILE CSV BARU DIBUAT DAN SIAP DIUNDUH.")
    print(f"📦 Mengkompresi {len(file_output_terbuat)} file ke dalam {NAMA_FILE_ZIP}...")

    # Membuat file ZIP dan menambahkan semua file hasil ke dalamnya
    with zipfile.ZipFile(NAMA_FILE_ZIP, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for filename in file_output_terbuat:
            zipf.write(filename)

    print(f"✅ Kompresi selesai. Sekarang mengunduh file {NAMA_FILE_ZIP}...")

    # Unduh file ZIP
    try:
        files.download(NAMA_FILE_ZIP)
        print(f"⬇️ Memicu unduhan untuk '{NAMA_FILE_ZIP}'. Periksa folder Downloads Anda.")
    except Exception as e:
        print(f"❌ Gagal mengunduh {NAMA_FILE_ZIP}. Error: {e}")

print("PROSES KOLEKTIF SELESAI.")

--- Skrip Modifikasi CSV (Filter Interval & Kolom Time) ---
Filter Counter: [1000n + 301, 1000n + 600]
----------------------------------------------------------------------
Ditemukan 20 file CSV untuk diproses: ['SPLIT_1001_AKHIR_Sample 1 Grade 4.csv', 'SPLIT_1001_AKHIR_Sample 4 Grade 5.csv', 'SPLIT_1001_AKHIR_Sample 5 Grade 3.csv', 'SPLIT_1001_AKHIR_Sample 2 Grade 2.csv', 'SPLIT_1001_AKHIR_Sample 1 Grade 3.csv', 'SPLIT_1001_AKHIR_Sample 5 Grade 5.csv', 'SPLIT_1001_AKHIR_Sample 4 Grade 2.csv', 'SPLIT_1001_AKHIR_Sample 1 Grade 5.csv', 'SPLIT_1001_AKHIR_Sample 3 Grade 2.csv', 'SPLIT_1001_AKHIR_Sample 5 Grade 4.csv', 'SPLIT_1001_AKHIR_Sample 4 Grade 3.csv', 'SPLIT_1001_AKHIR_Sample 2 Grade 5.csv', 'SPLIT_1001_AKHIR_Sample 2 Grade 3.csv', 'SPLIT_1001_AKHIR_Sample 4 Grade 4.csv', 'SPLIT_1001_AKHIR_Sample 1 Grade 2.csv', 'SPLIT_1001_AKHIR_Sample 3 Grade 5.csv', 'SPLIT_1001_AKHIR_Sample 3 Grade 4.csv', 'SPLIT_1001_AKHIR_Sample 3 Grade 3.csv', 'SPLIT_1001_AKHIR_Sample 5 Grade 2.csv', 'SPLIT_1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Memicu unduhan untuk 'hasil_filter_time_csv.zip'. Periksa folder Downloads Anda.
PROSES KOLEKTIF SELESAI.
